In [ ]:
import sys

sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from collections import defaultdict
from uuid import uuid4
import os

from pyvis.network import Network
from HUGS.ObjectStore import query_store

from HUGS.Modules import ObsSurface

%load_ext autoreload
%autoreload 2

In [ ]:
previous_path = os.environ["OPENGHG_PATH"]

os.environ["OPENGHG_PATH"] = "/tmp/vis_store_tmp"

# AGAGE
data_filepath = "../../../tests/data/proc_test_data/GC/capegrim-medusa.18.C"
prec_filepath = "../../../tests/data/proc_test_data/GC/capegrim-medusa.18.precisions.C"

results = ObsSurface.read_file((data_filepath, prec_filepath), data_type="GCWERKS", network="AGAGE", overwrite=True)

data_filepath = "../../../tests/data/proc_test_data/GC/trinidadhead.01.C"
prec_filepath = "../../../tests/data/proc_test_data/GC/trinidadhead.01.precisions.C"

results = ObsSurface.read_file((data_filepath, prec_filepath), data_type="GCWERKS", network="AGAGE", overwrite=True)

# CRDS
data_filepath = "../../../tests/data/proc_test_data/CRDS/tac.picarro.1minute.100m.test.dat"
results = ObsSurface.read_file(data_filepath, data_type="CRDS", network="DECC", overwrite=True)

# ICOS
data_filepath = "../../../tests/data/proc_test_data/ICOS/tta.co2.1minute.222m.min.dat"
results = ObsSurface.read_file(data_filepath, data_type="ICOS", network="ICOS", overwrite=True)


Get the data from the store

In [ ]:
data = query_store()

Create the network and add a node and some edges to it

In [ ]:
net = Network("800px", "100%", notebook=True, heading="OpenGHG Object Store")
# Set the physics layout of the network
# net.barnes_hut(gravity=-5000)
net.force_atlas_2based()

Create the ObsSurface node

In [ ]:
net.add_node(0, label="ObsSurface", color="#4e79a7", value=5000)

In [ ]:
nested_dict = lambda: defaultdict(nested_dict)
network_split = nested_dict()

for key, value in data.items():
    # Iterate over Datasources to select the networks
    network = value["network"]
    site = value["site"]
    network_split[network][site][key] = value

Then create a node for each network

In [ ]:
for network, sites in network_split.items():
    network_name = network.upper()
    net.add_node(network, label=network_name, color="#59a14f", value=2500)
    net.add_edge(source=0, to=network)
    
    # Then we want a subnode for each site  
    for site, data in sites.items():
        # Don't want to use a site here as a site might be in multiple networks
        site_name = site.upper()
        site_id = str(uuid4())
        net.add_node(site_id, label=site_name, color="#e15759", value=1000)
        net.add_edge(source=network, to=site_id)
        
        # Now for each site create the datasource nodes
        for uid, datasource in data.items():
            species = datasource["species"]
            instrument = datasource["instrument"].upper()

            label = f"{site.upper()} {species.upper()} {instrument}"
            title = f"\n".join([f"Site: {site.upper()}", f"Species : {species.upper()}", f"Instrument: {instrument}"])
            net.add_node(uid, label=label, title=title, color="#f28e2b", value=100)
            net.add_edge(source=site_id, to=uid)

Then create a node for each site

In [ ]:
net.show("openghg_objstore.html")

Restore the path if you're using a local object store

In [ ]:
os.environ["OPENGHG_PATH"] = previous_path